In [1]:
import pandas as pd
from libs import preprocessa, modelos, vo_X, metricas
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
random_state=26

In [2]:
#Se estiver rodando localmente 
caminhoDados="dados/"

#Esse json é gerado no notebook TrataSegmentosAnotam.ipynb
df  = pd.read_json(caminhoDados+'dfAlvoAnota.json') 
vocabulario = df["textoTotal"]


In [3]:
#Outras
df.loc[df['idTipoAnotacao'].isin([2,5,6,11,14,16,20,55,12]),['idTipoAnotacao']] = 99
#União Alt e inc
df.loc[df['idTipoAnotacao'] == 3, ['idTipoAnotacao']] = 1    

In [4]:
#df.groupby(['idTipoAnotacao','nomeTipoAnotacao']).idSegmento.count()
#Inclusão por republicação parcial 28, Prorrogação de Vigência 42,Renumeração 18, Sem Eficácia, 4 Execução Suspensa, 6 Revigoramento, 3 Regulamentação 3
#155
#(155/20302)*100
#df['idTipoAnotacao'].unique()

#df.groupby(['idTipoAnotacao','nomeTipoAnotacao']).idSegmento.count()
#filtrar as classes mais relevantes a partir de 0.5% de presença
#classesInteresse = [1,3,9,22,24,13,10,4,25]

In [5]:

def classificar (listaDados, classificaLSVC,classificaMNB):
    resultadoGeral = []
    for dados in listaDados:
        #if dados.estrategia == 'Segmento+Ementa Smote':
        classificadores = modelos.classificadores()
        classificadores.LSVC = classificaLSVC
        classificadores.MNB = classificaMNB
        resultadosClassificacao =  classificadores.classificar(dados.x,dados.y)
        for res in resultadosClassificacao:
            res.insert(1, dados.estrategia) 
        resultadoGeral.extend(resultadosClassificacao)
        
    df_resultado  = pd.DataFrame(resultadoGeral)
    df_resultado.columns = ['Modelo','Atributo',
                    'Acurácia','ACCDP','Precisão',
                    'DP', 'Revocação','DP', 'F1','F1DP' ]
    df_resultado.loc[df_resultado['Modelo']=='LinearSVC','Modelo']='SVM'
    df_resultado.loc[df_resultado['Modelo']=='MultinomialNB','Modelo']='Naive Bayes'
    return(df_resultado)  

def convertAnotacaoLabel(listY):
    anotacao = {1:'Alteração-inclusão',
    4:'Prorrogação',
    9:'Revogação',
    10:'Anulação',
    13:'Sem Efeito',
    22:'Retificação',
    24:'Vide',
    25:'Republicação',
    99:'Outras'
    }

    yLabel= []
    for y in listY:
        yLabel.append(anotacao.get(y))
    return yLabel
       

In [6]:
##Atributos iniciais
y = df['idTipoAnotacao']

dadosOriginais = []

vocabulario = df["textoIntegraLimpo"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["textoIntegraLimpo"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"Segmento"))

vocabulario = df["verbosTextoIntegra"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["verbosTextoIntegra"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"VerbosSegmento"))

vocabulario = df["textoTotal"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["textoTotal"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"Segmento+Ementa"))

vocabulario = df["verbos"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["verbos"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"TodosVerbos"))


In [7]:
## Seleção Atributos
dadosChi = []
for dados in dadosOriginais:
    xChi1 = preprocessa.seleciona(vocabulario,dados.x, y,0.1)
    dadosChi.append(vo_X.vo_X(xChi1,y, dados.estrategia + " Chi 0.1"))
    
    xChi25 = preprocessa.seleciona(vocabulario,dados.x, y,0.25)
    dadosChi.append(vo_X.vo_X(xChi25,y, dados.estrategia + " Chi 0.25"))
    
    xChi50 = preprocessa.seleciona(vocabulario,dados.x, y,0.5)
    dadosChi.append(vo_X.vo_X(xChi50,y, dados.estrategia + " Chi 0.5"))
    
   

In [8]:
#Reamostragem Smote
smote = SMOTE(random_state=random_state)
dadosSmote = []
for dados in dadosOriginais:     
    xs, ys = smote.fit_sample(dados.x,y)
    dadosSmote.append(vo_X.vo_X(xs, ys, dados.estrategia + " Smote"))
    
for dados in dadosChi:     
    xs, ys = smote.fit_sample(dados.x,y)
    dadosSmote.append(vo_X.vo_X(xs, ys, dados.estrategia + " Smote"))

In [9]:
#Reamostragem NearMiss
sampler = NearMiss()
dadosNearMiss = []

for dados in dadosOriginais: 
    xn, yn = sampler.fit_resample(dados.x,y)
    dadosNearMiss.append(vo_X.vo_X(xn, yn, dados.estrategia + " NearMiss"))

for dados in dadosChi:
    xn, yn = sampler.fit_resample(dados.x,y)
    dadosNearMiss.append(vo_X.vo_X(xn, yn, dados.estrategia + " NearMiss"))


In [10]:
listaDados = [*dadosOriginais, *dadosChi, *dadosSmote, *dadosNearMiss]

In [11]:
df_resultado = classificar(listaDados, True, True)
df_resultado.to_csv("resultado.csv")

In [12]:
#Variação parâmetro C
from sklearn.svm import LinearSVC
for dado in dadosSmote:
    if dado.estrategia == 'Segmento+Ementa Smote':
        model =  LinearSVC(random_state=26, max_iter=1000, penalty='l2', C=1)
        accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1 = metricas.calculaMetricasCV(model,dado.x, dado.y)
print('C1', accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1)


for dado in dadosSmote:
    if dado.estrategia == 'Segmento+Ementa Smote':
        model =  LinearSVC(random_state=26, max_iter=1000, penalty='l2', C=1.5)
        accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1 =  metricas.calculaMetricasCV(model,dado.x, dado.y)
print('C1.5',accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1)

for dado in dadosSmote:
    if dado.estrategia == 'Segmento+Ementa Smote':
        model =  LinearSVC(random_state=26, max_iter=1000, penalty='l2', C=2)
        accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1 = metricas.calculaMetricasCV(model,dado.x, dado.y)
print('C2', accuracy,std_acc, precision,std_pre, recall, std_rec,f1,std_f1)

C1 98.95 0.007 99.00999999999999 0.0057 98.95 0.007 98.94 0.0072
C1.5 98.98 0.007 99.03999999999999 0.0057 98.98 0.007 98.97 0.0072
C2 99.0 0.0069 99.06 0.0056 99.0 0.0069 98.99 0.007


In [13]:
#MAtriz de Confusão melhor modelo
from sklearn.model_selection import cross_val_predict
from sklearn.svm import LinearSVC
from libs import metricas

modelo = LinearSVC(random_state=26, max_iter=1000, C=2)

classes = ['Alteração-inclusão','Revogação','Vide', 'Prorrogação', 'Retificação','Anulação',
              'Republicação','Sem Efeito','Outras']


#Melhor Modelo obtido
for dado in dadosSmote:
    if dado.estrategia == 'Segmento+Ementa Smote':
        y_pred = cross_val_predict(modelo, dado.x,dado.y, cv=10)
        y_real = dado.y


y_realLabel = convertAnotacaoLabel(y_real)
y_predLabel = convertAnotacaoLabel(y_pred) 
matrix = metricas.matrixConfusa(y_realLabel, y_predLabel, classes)
matrix          



,Alteração-inclusão,Revogação,Vide,Prorrogação,Retificação,Anulação,Republicação,Sem Efeito,Outras
Alteração-inclusão,12559,485,64,6,81,0,141,0,61
Revogação,17,13333,16,0,0,28,2,0,1
Vide,0,30,13151,0,205,0,11,0,0
Prorrogação,0,0,0,13377,0,0,0,0,20
Retificação,1,0,15,0,13366,0,0,0,15
Anulação,0,2,0,0,0,13395,0,0,0
Republicação,0,0,0,0,0,0,13397,0,0
Sem Efeito,0,0,0,0,0,0,0,13397,0
Outras,0,0,0,0,0,0,0,0,13397


In [14]:
#MAtriz de Confusão comparativa sem smote

for dado in dadosOriginais:
    if dado.estrategia == 'Segmento+Ementa':
        modelo = LinearSVC(random_state=26, max_iter=1000, C=2)
        y_pred = cross_val_predict(modelo, dado.x,dado.y, cv=10)
        y_real = dado.y
    
y_realLabel = convertAnotacaoLabel(y_real)
y_predLabel = convertAnotacaoLabel(y_pred) 
matrix = metricas.matrixConfusa(y_realLabel, y_predLabel, classes)
matrix          




,Alteração-inclusão,Revogação,Vide,Prorrogação,Retificação,Anulação,Republicação,Sem Efeito,Outras
Alteração-inclusão,12822,423,37,7,72,0,28,0,8
Revogação,529,2843,20,0,2,26,13,4,2
Vide,182,31,692,9,50,13,5,13,7
Prorrogação,7,0,10,96,0,0,0,0,2
Retificação,264,18,27,1,1314,0,9,1,4
Anulação,2,12,12,0,4,90,1,1,1
Republicação,98,14,15,2,11,1,92,0,0
Sem Efeito,5,1,13,0,1,3,0,176,1
Outras,45,6,14,9,7,0,1,7,66
